In [1]:
# https://quantumcomputing.stackexchange.com/questions/24050/how-to-implement-a-exponential-of-a-hamiltonian-but-non-unitary-matrix-in-qisk

# control gate: https://qiskit.org/documentation/stubs/qiskit.circuit.ControlledGate.html

# circuit https://www.nature.com/articles/s41598-022-17660-8
import numpy as np
import configparser
from sklearn.preprocessing import normalize
import collections

from qiskit.circuit import ControlledGate
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute, Aer, transpile
from qiskit.quantum_info.operators import Operator
from qiskit.extensions import UnitaryGate
from qiskit.tools.visualization import plot_histogram

from qiskit.synthesis import MatrixExponential
from qiskit.quantum_info import Operator
from qiskit.quantum_info import SparsePauliOp
from qiskit.opflow.list_ops import SummedOp
from qiskit.circuit import Parameter
from qiskit.opflow import I, X, Y, Z, H, CX, Zero, ListOp, PauliExpectation, PauliTrotterEvolution, CircuitSampler, MatrixEvolution, Suzuki, PauliSumOp

In [2]:
example = 'example3'
config = configparser.ConfigParser()
config.read('config.ini')
A = config[example]['A']
x = config[example]['x']
b = config[example]['b']
A = np.array([[float(number) for number in list_.replace('[', '').replace(']', '').split(',')] for list_ in A.split(',]')]).flatten()
A = A.reshape((int(np.sqrt(np.shape(A)[0])), int(np.sqrt(np.shape(A)[0])))).astype('complex')
x = np.array([float(number) for number in x.replace('[', '').replace(']', '').split(',')]).flatten()
b = np.array([float(number) for number in b.replace('[', '').replace(']', '').split(',')]).flatten()

In [3]:
# u, s, vh = np.linalg.svd(A, full_matrices=False)
# b = np.array([np.sin(np.pi/4)**2, np.sin(np.pi/4)**2, np.sin(np.pi/4)**2, np.sin(np.pi/4)**2]).T
# x = np.linalg.solve(A, b)
# print(x)

In [15]:
def get_gate(A, n):    
    pauli_op = PauliSumOp(SparsePauliOp.from_operator(A))
    phi = Parameter('ϕ')
    evolution_op = (phi * pauli_op).exp_i() # exp(-iϕA)
    trotterized_op = PauliTrotterEvolution(trotter_mode=Suzuki(order=2, reps=1)).convert(evolution_op).bind_parameters({phi: np.pi/n})
    #----control---------
    gate = trotterized_op.to_circuit()
    gate.name = f"e^(i*A*pi/{n})"
    gate.label = f"e^(i*A*np.pi/{n})"
    gate = gate.to_gate().control()
    #---------------------
    return gate

In [5]:
n_b = int(np.log2(len(b)))
n_ = 9 #3 is an optimal
n_ancilla = 1
n_cl = n_b#+1
# quantum circuit initialization
b_x = QuantumRegister(n_b, 'b and x')
eigs = QuantumRegister(n_, 'eigenvalues')
ancilla = QuantumRegister(n_ancilla, 'ancilla')
classical = ClassicalRegister(n_cl, 'classical')
qc = QuantumCircuit(b_x, eigs, ancilla, classical)
# b-vector state preparation
# for i in range(n_b):
#     qc.ry(np.pi/2, i)
qc.initialize(b, b_x)
qc.h(eigs)
qc.x(ancilla)
qc.barrier()
# Matrix exponentiation
for i in range(0, len(eigs)):
    gate = get_gate(A, 2**(i+1))
    qc.append(gate,[eigs[i], *b_x])
qc.barrier()
# Phase estimation
for j in range(len(eigs)-1, 0, -1):
    qc.h(eigs[j])
    for m in range(j):
        qc.crz(-np.pi/float(2**(j-m)), eigs[j], eigs[m])
qc.barrier()
# As I understood, we wncode ancilla qubit to be sure that result will be correct
for j in range(1, 1+n_):
    qc.cry(np.pi/(2**j), n_b+n_-j, n_b+n_)
qc.barrier()
# Inverse quantum Fourier transform
for j in range(len(eigs)-1, 0, -1):
    qc.h(eigs[j])
    for m in range(j):
        qc.crz(np.pi/float(2**(j-m)), eigs[j], eigs[m])
qc.barrier()
# Eigenvalues storing in the vecor b register
for i in range(len(eigs), 0, -1):
    gate = get_gate(A, -2**(i+1))
    qc.append(gate,[eigs[i-1], *b_x])
qc.barrier()
# qubits measurement. I do not measure the ancilla qubit
# qc.reverse_bits()
qc.measure(b_x, classical)
# qc.draw(output='mpl', style={'backgroundcolor': '#EEEEEE'})

In [6]:
simulator = Aer.get_backend('qasm_simulator')
circ = transpile(qc, simulator)
shots = 2048
result = execute(qc, backend=simulator, shots=shots).result()

# Calculated result

In [7]:
counts = result.get_counts()
probabilities = counts.copy()
probs_upd = {}
for k, v in probabilities.items():
    value = v/shots
    probabilities[k] = value

# Result vector

In [8]:
vect = np.array([v for k,v in probabilities.items()])
vect.sort()
np.round(vect, 5)

array([0.0083 , 0.00928, 0.01367, 0.01514, 0.01807, 0.02295, 0.08545,
       0.82715])

# Real normalized result

In [10]:
real_x_norm = (np.array(x) / np.linalg.norm(x))**2
real_x_norm.sort()
np.round(real_x_norm, 5)

array([0.     , 0.00109, 0.00182, 0.00634, 0.00844, 0.02139, 0.07319,
       0.88773])

# Error

In [11]:
mse = (np.square(vect - real_x_norm)).mean()
print(mse)
abs_err = np.mean([np.abs((v-r)/r) for v,r in zip(vect, real_x_norm)])
print(abs_err)

0.000533631236082668
502.7453264198555
